Import pandas for data manipultaion, Numpy for linear algebra, hashlib for masking.

In [1]:
import pandas as pd
import numpy as np
import hashlib

Read CSV files containing online data, Heritage data, Oso Creek data, and Windward data. Data can only be downloaded by campus or online so data sets are concatenated.

In [2]:
#Read data
mlc_Heritage_Online = pd.read_csv("/kaggle/input/mlc-heritage-data/Heritage-Online.csv")
mlc_Heritage_In_Person = pd.read_csv("/kaggle/input/mlc-heritage-data/Heritage-tutoring.csv")
mlc_Heritage_Online.dropna(inplace = True)

#Create data frames for each Campus
mlc_Oso = pd.read_csv("/kaggle/input/mlc-campuses-data/Oso-Student_Subject_Hours.csv")
mlc_Windward = pd.read_csv("/kaggle/input/mlc-campuses-data/Windward-Student_Subject_Hours.csv")
Heritage_dframes = [mlc_Heritage_Online, mlc_Heritage_In_Person]
mlc_Heritage = pd.concat(Heritage_dframes)

#Create a dataframe containing all data
frames = [mlc_Heritage, mlc_Oso,mlc_Windward]
mlc_all = pd.concat(frames)

#Replace spaces between variable names with underscore
mlc_all.columns = [c.replace(' ', '_') for c in mlc_all.columns]

#Change Student_Number variable to string
mlc_all['Student_Number'] = mlc_all['Student_Number'].astype(str)

# Apply hashing function to the column Student_Number for privacy
mlc_all['Student_Number_Hash'] = mlc_all['Student_Number'].apply(
    lambda x: 
        hashlib.sha256(x.encode()).hexdigest()
)

# Drop all sensitive data then display first five rows
mlc_all = mlc_all.drop(['First_Name', 'Middle_Name', 'Last_Name', 'Student_Number'],axis=1)
mlc_all.head()

,Subject_Area_Code_Prefix,Subject_Area_Code_Number,Course,Period_(hr:mins),Visits,Service,Campus,Student_Number_Hash
0,BUSI,2305.0,Business Statistics,1:48,3.0,online,online,d8af2c4478ce8b12348851a0172599a3a4db3dfea9437f...
2,CHEM,1411.0,General Inorganic Chemistry I,0:35,1.0,online,online,1743f8435790daf38290bd1f06bf315e02422089202a3c...
4,CHEM,1412.0,General Inorganic Chemistry II,0:28,1.0,online,online,ce908609356a1a4510237915bef7d1ccee1a5535b4cd0e...
5,DEBA,1.0,Developmental Mathematics Base Non-Course Base...,3:06,1.0,online,online,bfa51e7c0a40f5f3c8026c11baf6b597466f5b884e87b9...
6,DEBA,1.0,Developmental Mathematics Base Non-Course Base...,1:34,4.0,online,online,aad8c5f5e79617bd813ad9426d677bca4a7c831e46bb6e...


In [3]:
#Rename Columns
mlc_all.rename(columns = {'Subject_Area_Code_Prefix' : 'Course_Prefix',
                          'Subject_Area_Code_Number':'Course_Code', 
                          'Visits':'Student_Visits',
                          'Period_(hr:mins)':'Total_Time'},
                           inplace = True) 

#Some datasets had empty entries for Service column, fill them with correct data
mlc_all['Service'] = mlc_all['Service'].fillna('Tutoring')

#Drop any empty rows
mlc_all.dropna(inplace = True)

#Preview data
mlc_all.head()

,Course_Prefix,Course_Code,Course,Total_Time,Student_Visits,Service,Campus,Student_Number_Hash
0,BUSI,2305.0,Business Statistics,1:48,3.0,online,online,d8af2c4478ce8b12348851a0172599a3a4db3dfea9437f...
2,CHEM,1411.0,General Inorganic Chemistry I,0:35,1.0,online,online,1743f8435790daf38290bd1f06bf315e02422089202a3c...
4,CHEM,1412.0,General Inorganic Chemistry II,0:28,1.0,online,online,ce908609356a1a4510237915bef7d1ccee1a5535b4cd0e...
5,DEBA,1.0,Developmental Mathematics Base Non-Course Base...,3:06,1.0,online,online,bfa51e7c0a40f5f3c8026c11baf6b597466f5b884e87b9...
6,DEBA,1.0,Developmental Mathematics Base Non-Course Base...,1:34,4.0,online,online,aad8c5f5e79617bd813ad9426d677bca4a7c831e46bb6e...


The variable 'Course_Code' needs to be a 4 code with no decimal place. 

In [4]:
#Make 'Course_Code' an integer type
mlc_all['Course_Code'] = mlc_all['Course_Code'].astype(int)

#Make 'Student_Visits' an integer
mlc_all['Student_Visits'] = mlc_all['Student_Visits'].astype(int)

#Display
mlc_all.head()

,Course_Prefix,Course_Code,Course,Total_Time,Student_Visits,Service,Campus,Student_Number_Hash
0,BUSI,2305,Business Statistics,1:48,3,online,online,d8af2c4478ce8b12348851a0172599a3a4db3dfea9437f...
2,CHEM,1411,General Inorganic Chemistry I,0:35,1,online,online,1743f8435790daf38290bd1f06bf315e02422089202a3c...
4,CHEM,1412,General Inorganic Chemistry II,0:28,1,online,online,ce908609356a1a4510237915bef7d1ccee1a5535b4cd0e...
5,DEBA,1,Developmental Mathematics Base Non-Course Base...,3:06,1,online,online,bfa51e7c0a40f5f3c8026c11baf6b597466f5b884e87b9...
6,DEBA,1,Developmental Mathematics Base Non-Course Base...,1:34,4,online,online,aad8c5f5e79617bd813ad9426d677bca4a7c831e46bb6e...


'Course_Code' column should have entries with 4 digits, we see DEBA has entry 1 should be 0001.

In [5]:
#Format 'Course_Code' as 4 digit value with zeros in front of existing values
#___1 becomes 0001. 

mlc_all['Course_Code'] = mlc_all['Course_Code'].apply(lambda x: '{0:0>4}'.format(x))
mlc_all.head()

,Course_Prefix,Course_Code,Course,Total_Time,Student_Visits,Service,Campus,Student_Number_Hash
0,BUSI,2305,Business Statistics,1:48,3,online,online,d8af2c4478ce8b12348851a0172599a3a4db3dfea9437f...
2,CHEM,1411,General Inorganic Chemistry I,0:35,1,online,online,1743f8435790daf38290bd1f06bf315e02422089202a3c...
4,CHEM,1412,General Inorganic Chemistry II,0:28,1,online,online,ce908609356a1a4510237915bef7d1ccee1a5535b4cd0e...
5,DEBA,0001,Developmental Mathematics Base Non-Course Base...,3:06,1,online,online,bfa51e7c0a40f5f3c8026c11baf6b597466f5b884e87b9...
6,DEBA,0001,Developmental Mathematics Base Non-Course Base...,1:34,4,online,online,aad8c5f5e79617bd813ad9426d677bca4a7c831e46bb6e...


The time_string_to_decimals(time_string) function receives a string time value in the format hh:mm:ss or similar, splits at each ':', then returns a float value as hours. 

In [6]:
def time_string_to_decimals(time_string):
    fields = time_string.split(":")
    hours = fields[0] if len(fields) > 0 else 0.0
    minutes = fields[1] if len(fields) > 1 else 0.0
    seconds = fields[2] if len(fields) > 2 else 0.0
    return float(hours) + (float(minutes) / 60.0) + (float(seconds) / pow(60.0, 2))

In [7]:
#Change "Total_Time" and "Course_Prefix" to string values.
mlc_all[['Total_Time', 'Course_Prefix']] = mlc_all[['Total_Time', 'Course_Prefix']].astype(str)

Update dataset containing all data by applying the time_string_to_decimals function to every entry in the "Total_Time" column.

In [8]:
mlc_all['Total_Time'] = mlc_all['Total_Time'].apply(time_string_to_decimals)
mlc_all.head()

,Course_Prefix,Course_Code,Course,Total_Time,Student_Visits,Service,Campus,Student_Number_Hash
0,BUSI,2305,Business Statistics,1.800000,3,online,online,d8af2c4478ce8b12348851a0172599a3a4db3dfea9437f...
2,CHEM,1411,General Inorganic Chemistry I,0.583333,1,online,online,1743f8435790daf38290bd1f06bf315e02422089202a3c...
4,CHEM,1412,General Inorganic Chemistry II,0.466667,1,online,online,ce908609356a1a4510237915bef7d1ccee1a5535b4cd0e...
5,DEBA,0001,Developmental Mathematics Base Non-Course Base...,3.100000,1,online,online,bfa51e7c0a40f5f3c8026c11baf6b597466f5b884e87b9...
6,DEBA,0001,Developmental Mathematics Base Non-Course Base...,1.566667,4,online,online,aad8c5f5e79617bd813ad9426d677bca4a7c831e46bb6e...


In [9]:
#Round 'Total_Time' column to hundredths place
decimals = 2    
mlc_all['Total_Time'] = mlc_all['Total_Time'].apply(lambda x: round(x, decimals))
mlc_all.head()

,Course_Prefix,Course_Code,Course,Total_Time,Student_Visits,Service,Campus,Student_Number_Hash
0,BUSI,2305,Business Statistics,1.80,3,online,online,d8af2c4478ce8b12348851a0172599a3a4db3dfea9437f...
2,CHEM,1411,General Inorganic Chemistry I,0.58,1,online,online,1743f8435790daf38290bd1f06bf315e02422089202a3c...
4,CHEM,1412,General Inorganic Chemistry II,0.47,1,online,online,ce908609356a1a4510237915bef7d1ccee1a5535b4cd0e...
5,DEBA,0001,Developmental Mathematics Base Non-Course Base...,3.10,1,online,online,bfa51e7c0a40f5f3c8026c11baf6b597466f5b884e87b9...
6,DEBA,0001,Developmental Mathematics Base Non-Course Base...,1.57,4,online,online,aad8c5f5e79617bd813ad9426d677bca4a7c831e46bb6e...
